In [1]:
import ee
import os
from pathlib import Path
from datetime import datetime as dt
from abf import abf

In [2]:
__version__ = ""
exec(Path(f"{os.getcwd()}/src/abf/version.py").read_text(encoding="utf-8"))

In [3]:
# Initialize Google Earth Engine
# ee.Authenticate() (might be needed in some situations)
ee.Initialize()

In [4]:
# Data path
folderRoot = os.getcwd() + "/results"
if not os.path.exists(folderRoot):
    os.mkdir(folderRoot)

# Images path
folderCache = os.getcwd() + "/cache"
if not os.path.exists(folderCache):
    os.mkdir(folderCache)
    
# Selected image date to dectection
selectedDate = "2019-06-03"

# folder to save results from algorithm at
folder = (
    folderRoot
    + "/"
    + dt.now().strftime("%Y%m%d_%H%M%S")
    + f"[v={str(__version__)}-sandbox,d={selectedDate},t=180,m=ocsvm,s=modis]")
if not os.path.exists(folder):
    os.mkdir(folder)

In [5]:
# fix permissions
!chmod 777 $folderRoot
!chmod 777 $folder
!chmod 777 $folderCache

zsh:1: no matches found: /Users/phmananias/Documents/unesp/abf/results/20221210_115540[v=v0.34.0-sandbox,d=2019-06-03,t=180,m=ocsvm,s=modis]


In [6]:
# Latitude and Longitude (PoI)
lat_lon = "-83.48811946836814,41.85776095627803,-83.18290554014548,41.677617395337826"

# (colon, lat and lon separated by comma, all together)
# and dates by the user (two dates, beginning and end, separated by commas)
x1, y1, x2, y2 = lat_lon.split(",")

# Assemble Geometry on Google Earth Engine
geometry = ee.Geometry.Polygon(
    [
        [
            [float(x1), float(y2)],
            [float(x2), float(y2)],
            [float(x2), float(y1)],
            [float(x1), float(y1)],
            [float(x1), float(y2)],
        ]
    ]
)

In [7]:
# create algorithm object
algorithm = abf.Abf(days_threshold=180,
                    grid_size=12,
                    sensor="modis", 
                    geometry=geometry,
                    lat_lon=lat_lon,
                    cache_path=folder,
                    force_cache=False,
                    days_in=5,
                    days_out=5,
                    from_date="2019-07-11",
                    model="rf",
                    reducer=True,
                    class_mode=True,
                    attribute_lat_lon=False)


Selected sensor: Modis MOD09GA.006 [GEE Dataset = MODIS/006/MOD09GA]
Statistics: scale=500 meters, pixels=2720, initial_date='2019-01-12', end_date='2019-07-11', interval_images='180', interval_unique_images='180', water_mask_images='16', grid_size='11', days_in='5', days_out='5', morph_op='None', morph_op_iters='1', convolve='False', convolve_radius='1', scaler='minmax', model='rf', fill_missing='time', reducer='True', normalized='True', class_mode='True', class_weight='False', propagate='False', rs_train_size='500', rs_iter='500', pca_size='0.9', attr_lat_lon='False', attr_doy='True'


In [8]:
# preprocessing
algorithm.process_timeseries_data()
algorithm.process_training_data(df=algorithm.df_timeseries)


Starting time series processing ...
Trying to extract it from the cache...
Error trying to get it from cache: either doesn't exist or is corrupted! Creating it again...
Processing date [2019-01-12]...
Extracting geometry (0) 1 of 1...
Pixel and cloud score for geometry #1: 1.0 and 0.0!
Image is not good for processing and it was discarded!
Processing date [2019-01-13]...
Extracting geometry (0) 1 of 1...
Pixel and cloud score for geometry #1: 1.0 and 0.0!
Image is not good for processing and it was discarded!
Processing date [2019-01-14]...
Extracting geometry (0) 1 of 1...
Pixel and cloud score for geometry #1: 1.0 and 0.82484!
   pixel index row column        date doy                 lat  \
0      0     0   0      0  2019-01-14   0  -83.18624109767799   
1      1     0   0      0  2019-01-14   0  -83.18624109767799   
2      2     0   0      0  2019-01-14   0  -83.18624109767799   
3      3     0   0      0  2019-01-14   0  -83.18624109767799   
4      4     0   0      0  2019-01-14

In [ ]:
# train/predict
algorithm.train(batch_size=2048, disable_gpu=True)
algorithm.predict(folder=folder+"/prediction")


Starting the training process...

RandomizedSearchCV datasets length: rs=((500, 5), (500, 5))

RandomizedSearchCV datasets frequency:
[[   0 2439]
 [   4   61]]

Creating the Random Forest Regressor/Classifier with RandomizedSearchCV parameterization model...
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 t


Evaluating the RF (n_estimators=1880,max_features=0.75,max_depth=20,min_samples_leaf=2,min_samples_split=20,bootstrap=True) model...


[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done 1880 out of 1880 | elapsed:    2.4s finished


Report for the RF (n_estimators=1880,max_features=0.75,max_depth=20,min_samples_leaf=2,min_samples_split=20,bootstrap=True) model: Acc:0.999, BAcc:0.9906, F1-Score:0.999, Kappa:0.9789, vKappa:0.0, Tau:0.9789, vTau:0.0, p-value:0.0, Mcc:0.9789, RMSE:0.1241, MAE:0.0039, R^2:0.9576, TP:3535, FP:83, TN:151066, FN:66
finished!

Starting the predicting process...

Extracting images (5) that it will be used in the prediction process
Processing date [2019-07-06]...
Extracting geometry (0) 1 of 1...
Pixel and cloud score for geometry #1: 1.0 and 0.1862!
   pixel index row column        date doy                 lat  \
0      0     0   0      0  2019-07-06   0  -83.18624109767799   
1      1     0   0      0  2019-07-06   0  -83.18624109767799   
2      2     0   0      0  2019-07-06   0  -83.18624109767799   
3      3     0   0      0  2019-07-06   0  -83.18624109767799   
4      4     0   0      0  2019-07-06   0  -83.18624109767799   

                  lon cloud                   mndwi  ...  

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done 1880 out of 1880 | elapsed:    1.5s finished



Starting the prediction for RF (n_estimators=1880,max_features=0.75,max_depth=20,min_samples_leaf=2,min_samples_split=20,bootstrap=True) model...

[Pixel] Evaluating prediction for date 2019-07-11...
Acc:0.9937, BAcc:0.9782, F1-Score:0.9938, Kappa:0.9142, vKappa:0.0008, Tau:0.9152, vTau:0.0, p-value:0.0, Mcc:0.9152, RMSE:0.3178, MAE:0.0252, R^2:0.8204, TP:50, FP:7, TN:1367, FN:2

[Pixel] Evaluating prediction for date 2019-07-12...
Acc:0.8738, BAcc:0.7475, F1-Score:0.8579, Kappa:0.5934, vKappa:0.001, Tau:0.6453, vTau:0.0005, p-value:0.0, Mcc:0.6453, RMSE:1.4209, MAE:0.5048, R^2:0.3217, TP:116, FP:2, TN:708, FN:117

[Pixel] Evaluating prediction for date 2019-07-13...
Acc:0.9362, BAcc:0.9326, F1-Score:0.9357, Kappa:0.8711, vKappa:0.0013, Tau:0.8784, vTau:0.0013, p-value:0.0, Mcc:0.8784, RMSE:1.0106, MAE:0.2553, R^2:0.744, TP:77, FP:0, TN:99, FN:12

[Pixel] Evaluating prediction for date 2019-07-14...
Acc:0.9571, BAcc:0.7947, F1-Score:0.9526, Kappa:0.7087, vKappa:0.0013, Tau:0.7323, vTa

In [ ]:
# prediction results
algorithm.save_dataset(df=algorithm.df_results, path=folder+'/results.csv')

In [ ]:
# preprocessing results
algorithm.save_dataset(df=algorithm.df_timeseries, path=folder+'/timeseries.csv')